# 1. Threading

- Python interpreter executes code in a line-by-line fashion.  
- This means it cannot move ahead unless it is done executing a line.  
- But what if you need to **simultaneously run more than one code execution** within a single script?  
- A **thread** is a flow of execution. It allows your program to run more than one independent task at once, using **threading**.  
- However, depending on the Python implementation, it may or may not support true parallel threading but can appear to run simultaneously.  
- **`threading`** is a built-in Python library that allows working with threads
#### Thread Method
- `.start()` → Initiates the thread and runs the respective method.  
- `.join()` → Waits for the thread to finish before continuing execution.
#### Advantages and Cautions of Threading
- ✅ Threading allows your program to run multiple tasks simultaneously without immediate error breaks.  
- ⚠️ If used **recklessly**, threading may reduce speed, efficiency, and code clarity.  
- **Key takeaway:** Use threading only where necessary and with caution.  


## 1.2 Example 1
- We can create two methods:  
  1. `get_cubic()` → Cubes the input.  
  2. `get_inverse()` → Inverts the input.  
- Using the **Thread** class, we pass the method and its respective arguments.  
- This lets both methods run simultaneously.  


In [6]:
import threading
import datetime

def get_cube(num):
    print(datetime.datetime.now())
    print("Cube of ", num, " is: ", (num * num * num))
    print("")
    

def get_inverse(num):
    print(datetime.datetime.now())
    print("Inverse of ", num, " is: ", (1/num))

    
if __name__ == '__main__':
    # Creating Threads
    t1 = threading.Thread(target=get_cube, args=(5,))
    t2 = threading.Thread(target=get_inverse, args=(1545,))

    # Starting Threads
    t1.start()
    t2.start()

    # Waiting for each to finish
    t1.join()  # t1 waits until t2 executes


    print("Finish Threading!")

2025-09-17 17:58:21.138949
Cube of  5  is:  125

2025-09-17 17:58:21.139673
Inverse of  1545  is:  0.0006472491909385113
Finish Threading!


### 1.2.1 Example 1 with some syntax changes

In [7]:
import threading
import datetime

def get_cube(num):
    print(datetime.datetime.now())
    print("Cube is: {}" .format(num * num * num))
    print("")
    

def get_inverse(num):
    print(datetime.datetime.now())
    print("Inverse is: {}" .format(1/num))

    
if __name__ == '__main__':
    # Creating Threads
    t1 = threading.Thread(get_cube(67))
    t2 = threading.Thread(get_inverse(15))

    # Starting Threads
    t1.start()
    t2.start()

    # Waiting for each to finish
    t2.join()  # t2 waits until t1 executes


    print("Finish Threading!")

2025-09-17 17:59:28.636428
Cube is: 300763

2025-09-17 17:59:28.636867
Inverse is: 0.06666666666666667
Finish Threading!


## 1.3 Example 2
- If `10` (a number) is passed, it works fine.  
- If `'10'` (a string) is passed to `get_inverse()`, it will throw an error.  
- Since threading is used, the program does not completely crash—it will continue execution after handling the thread’s error.  

In [9]:
import threading
import datetime

def get_cube(num):
    print(datetime.datetime.now())
    print("Cube is: {}" .format(num * num * num))
    print("")
    

def get_inverse(num):
    print(datetime.datetime.now())
    print("Inverse is: {}" .format(1/num))

    
if __name__ == '__main__':
    # Creating Threads
    t1 = threading.Thread(get_cube(67))
    t2 = threading.Thread(get_inverse('15'))

    # Starting Threads
    t1.start()
    t2.start()

    # Waiting for each to finish
    t1.join()  # t1 waits until t2 executes
    t2.join()  # t2 waits until t1 executes


    print("Finish Threading!")

2025-09-17 18:01:55.399540
Cube is: 300763

2025-09-17 18:01:55.400554


TypeError: unsupported operand type(s) for /: 'int' and 'str'

## 1.4 Exercise 1

You are required to conduct an analysis on **temperature readings** from a plant.  

- Dataset: **`temperatures.csv`**  
- Tasks:  

| Task No. | Required Calculation                          |
|----------|-----------------------------------------------|
| 1        | Average time lapse in readings                |
| 2        | Average per hour temperature recording        |
| 3        | Largest spike in data (percentage)            |
| 4        | Overall deviation in the data                 |

- Conduct the above **4 calculations using two threads**:  
  - Thread 1 → Task 1 & 2  
  - Thread 2 → Task 3 & 4  
- Measure execution time **with and without threads**.  
- Log results in: **`section_four/temperature.txt`** using the appropriate library.  

In [14]:
import logging
import time
import datetime 
import threading 
import csv 

# read file from csv 
file = "3.1_temperature.csv"

# read data 
temp_list = []
time_list = []
with open(file, 'r', encoding="utf-8-sig") as f:  # utf-8-sig removes BOM
    reader = csv.reader(f)
    for row in reader:
        time_check, temp = row
        time_check = datetime.datetime.strptime(time_check.strip(), '%H:%M:%S')
        temp = float(temp)
        temp_list.append(temp)
        time_list.append(time_check)

# define method for time lapse 
def caclulate_time_lapse(time_data):
    assert type(time_data) == list 
    all_time_lapses = []
    for ind in range(1, len(time_data)):
        lapse = (time_data[ind] - time_data[ind-1]).total_seconds()
        all_time_lapses.append(lapse) 
    avg_time_lapse = sum(all_time_lapses)/len(all_time_lapses)
    return avg_time_lapse

# define method for largest spike in temperature 
def calculate_largest_spike(temperature_data):
    pct_change = []
    for ind in range(1, len(temperature_data)):
        diff = (temperature_data[ind] - temperature_data[ind-1]) / temperature_data[ind-1]
        pct_change.append(diff)
    return max(pct_change)

# define method for calculating hourly temperature 
def calculate_hourly_temperature(time_data, temp_data):
    time_in_hours = [time_val.hour for time_val in time_data]
    unique_hours = set(time_in_hours)   
    avg_readings = []
    for unique_hour in unique_hours:
        hour_readings = [temp_data[ind] for ind in range(len(time_in_hours)) if time_in_hours[ind] == unique_hour]
        avg = sum(hour_readings) / len(hour_readings)
        avg_readings.append(round(avg, 2))
    return avg_readings

# define method for calculating standard deviation in temperature 
def calculate_deviation_temperature(temperature_data):
    avg_temperature = sum(temperature_data) / len(temperature_data)
    total_observations = len(temperature_data)
    sq_diff = [(value - avg_temperature) ** 2 for value in temperature_data]
    sum_squared_avg = sum(sq_diff) / total_observations
    standard_deviation = sum_squared_avg ** 0.5
    return standard_deviation

# define method for first thread calculations
def first_thread_function():
    tl = caclulate_time_lapse(time_list)
    print('The time lapse is:', tl)
    ls = calculate_largest_spike(temp_list)
    print('The largest spike in temperature is:', ls)
    return 

# define method for second thread calculations 
def second_thread_function():
    av_temp = calculate_hourly_temperature(time_list, temp_list)
    print('Average hourly temperatures are ', av_temp)
    std = calculate_deviation_temperature(temp_list)
    print('Deviation in temperature is: ', std)
    return 

def get_time(start, end):
    return round(end - start, 10)

def main():
    # with threading
    start = time.time()
    t1 = threading.Thread(target=first_thread_function) 
    t2 = threading.Thread(target=second_thread_function) 
    t1.start(); t2.start()
    t1.join(); t2.join()
    end = time.time()
    print('With threading it took: {} seconds.'.format(get_time(start, end)))
    print("")

    # without threading
    start2 = time.time()
    first_thread_function()
    second_thread_function()
    end2 = time.time()
    print('Without threading it took: {} seconds'.format(get_time(start2, end2)))

main()


The time lapse is: 562.5
The largest spike in temperature is: 1.3157894736842104
Average hourly temperatures are  [21.01, 16.83, 15.7]
Deviation in temperature is:  5.60796517075502
With threading it took: 0.0010330677 seconds.

The time lapse is: 562.5
The largest spike in temperature is: 1.3157894736842104
Average hourly temperatures are  [21.01, 16.83, 15.7]
Deviation in temperature is:  5.60796517075502
Without threading it took: 0.000138998 seconds
